In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
from pyutils import *
from matplotlib.patches import Circle
import plotting as pp
from dataloc import *
import catalog_definitions as cat
from groupcatalog import deserialize, serialize, GroupCatalog, SDSSGroupCatalog


%load_ext autoreload
%autoreload 2

In [ ]:
DPI = 80
pp.DPI = DPI

After the group finder is run, this notebook is used to post pp.process the results, generating plots and such for analysis.


## Loading existing datasets 

In [ ]:
mxxl_all=deserialize(cat.mxxl_all)
mxxl_fiberonly=deserialize(cat.mxxl_fiberonly)
mxxl_nn=deserialize(cat.mxxl_nn)
mxxl_simple_4=deserialize(cat.mxxl_simple_4)

In [ ]:
sdss_vanilla = deserialize(cat.sdss_vanilla)
sdss_colors = deserialize(cat.sdss_colors)
sdss_colors_chi = deserialize(cat.sdss_colors_chi)

In [ ]:
fiberonly_BGS=deserialize(cat.bgs_fiberonly)
fiberonly_1pass_BGS=deserialize(cat.bgs_fiberonly_1pass)
nn_BGS=deserialize(cat.bgs_nn)
nn_BGS_sdsslike=deserialize(cat.bgs_nn_sdsslike)
simple2_BGS=deserialize(cat.bgs_simple_2)
simple2_BGS_c=deserialize(cat.bgs_simple_2_c)
simple4_BGS=deserialize(cat.bgs_simple_4)
simple4_BGS_c=deserialize(cat.bgs_simple_4_c)
bgs_simple_4_no_sdss=deserialize(cat.bgs_simple_4_no_sdss)
bgs_simple_4_4p = deserialize(cat.bgs_simple_4_4p)

In [ ]:
all_u = deserialize(cat.uchuu_all)

## View Plots

In [ ]:
pp.plots(bgs_simple_4_no_sdss, simple4_BGS, bgs_simple_4_4p, sdss_vanilla)

In [ ]:
from groupcatalog import mstar_vmax_weighted
print(np.sum(np.isnan(simple4_BGS.all_data.mstar)) / len(simple4_BGS.all_data.mstar))
print(np.sum(simple4_BGS.all_data.z_assigned_flag) / len(simple4_BGS.all_data.z_assigned_flag))

In [ ]:
pp.plots(simple4_BGS)

In [ ]:
pp.compare_fsat_color_split(sdss_vanilla, nn_BGS_sdsslike)
#pp.compare_fsat_color_split(all, simple_2)
#pp.compare_fsat_color_split(all, simple_4)
pp.compare_fsat_color_split(sdss_vanilla, simple4_BGS)


In [ ]:
pp.qf_cen_plot(simple4_BGS)

In [ ]:
pp.fsat_by_z_bins(simple4_BGS, z_bins=np.array([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 1.0]), show_plots=True)
#pp.fsat_by_z_bins(mxxl_simple_4, z_bins=np.array([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 1.0]), show_plots=False, aggregation=pp.fsat_truth_vmax_weighted)


In [ ]:
pp.fsat_by_z_bins(mxxl_simple_4)


In [ ]:
pp.plots(mxxl_all, mxxl_fiberonly, mxxl_nn, mxxl_simple_4)


In [ ]:
#pp.plots(simple4_BGS, simple2_BGS, nn_BGS)
#pp.plots(simple_4, truth_on=True)
pp.plots(mxxl_all, truth_on=True)
#pp.plots(deserialize(cat., nn_BGS, simple2_BGS, simple4_BGS)
#pp.plots(sdss_vanilla, nn_BGS_sdsslike)

In [ ]:
pp.plots(fiberonly_BGS, nn_BGS, simple4_BGS)


In [ ]:
pp.plots(sdss_vanilla, simple4_BGS)


In [ ]:
# Print out biggest group size
for dataset in [simple4_BGS, simple2_BGS, fiberonly_BGS, fiberonly_1pass_BGS, sdss_vanilla]:
    print(dataset.name)
    print(dataset.all_data.groupby('igrp').size().max())


In [ ]:
# SDSS Examine Bimodality

z=sdss_vanilla.all_data['z']
gmr=sdss_vanilla.all_data['Mag_g'] - sdss_vanilla.all_data['Mag_r']
junk=plt.hist(gmr, bins=np.linspace(-1,3,300), alpha=0.4)
#junk=plt.hist(k_correct(sdss_vanilla.all_data['Mag_g'], z, gmr, band='g')  - k_correct(sdss_vanilla.all_data['Mag_r'], z, gmr, band='r'), bins=500, alpha=0.4)
junk=plt.hist(sdss_vanilla.all_data['Dn4000'], bins=np.linspace(0,4,300), alpha=0.4)
plt.xlim(-1, 3)

## BGS and SDSS Target Overlap Analysis

TODO: need to use a version of SDSS data that doesn't have nearest-neighbor assigned redshifts in it!

In [ ]:
pd.options.mode.copy_on_write = True

# For this comparison, use pure NN BGS 
bgs_to_use = simple4_BGS.all_data
lost_bgs = bgs_to_use.loc[bgs_to_use['z_assigned_flag'] == 1]
sdss_cat = sdss_vanilla.all_data

catalog = coord.SkyCoord(ra=sdss_cat.RA.to_numpy()*u.degree, dec=sdss_cat.Dec.to_numpy()*u.degree, frame='icrs')
to_match = coord.SkyCoord(ra=lost_bgs.RA.to_numpy()*u.degree, dec=lost_bgs.Dec.to_numpy()*u.degree, frame='icrs')

idx, d2d, d3d = coord.match_coordinates_sky(to_match, catalog, nthneighbor=1, storekdtree=False)

# if angular distance is < 3", then we consider it a match to SDSS catalog
lost_bgs['sdss_z'] = np.where(d2d < 3*u.arcsec, sdss_cat.iloc[idx]['z'], np.nan)
lost_bgs_matched = lost_bgs.loc[~np.isnan(lost_bgs['sdss_z'])]
print(f"Matched {len(lost_bgs_matched)} out of {len(lost_bgs)} lost BGS galaxies to SDSS catalog, ({len(lost_bgs_matched)/len(lost_bgs)*100:.2f}%)")

good_match = np.isclose(lost_bgs_matched.z, lost_bgs_matched.sdss_z, atol=0.001).sum()
print(f"Good match: {good_match} out of {len(lost_bgs_matched)}, or {good_match/len(lost_bgs_matched)*100:.2f}%")

## Simulations-Only Analysis

### UCHUU Issues

In [ ]:
plt.hist(all_u.all_data['M_halo'], bins=pp.Mhalo_bins, alpha=0.4)
plt.hist(all_u.all_data['uchuu_halo_mass']*10**10, bins=pp.Mhalo_bins, alpha=0.4)
plt.loglog()

# TODO do we expect the mass distribution of halos to be so different from the UCHUU SHAM catalog and our assigned halo?

In [ ]:
# TODO 1 / VMax corrections do odd thing to UCHUU Truth. Why?
pp.hod_plots(all_u)

### What effect does Fiber Assignment have on the luminosity function?

In [ ]:
pp.group_finder_centrals_halo_masses_plots(mxxl_all, [mxxl_fiberonly, mxxl_simple_4])

### Purity and Completeness

In [ ]:
pp.test_purity_and_completeness(mxxl_all, mxxl_fiberonly, mxxl_nn, mxxl_simple_4)

In [ ]:
pp.purity_complete_plots(mxxl_all, mxxl_fiberonly, mxxl_nn, mxxl_simple_4)

### Compare halos to truth

In [ ]:
pp.assigned_halo_analysis(mxxl_simple_4)

### Compare assigned implied abs mags to truth from MXXL

In [ ]:
all_unobs_counts = mxxl_all.all_data[mxxl_all.all_data.z_assigned_flag].groupby('Lgal_bin').RA.count()
simple_4_ubobs_counts = mxxl_simple_4.all_data.groupby('Lgal_bin').RA.count()


In [ ]:
pp.L_func_plot([mxxl_all, mxxl_simple_4], [all_unobs_counts, simple_4_ubobs_counts])



#pp.L_func_plot([all, simple_4], [all.all_data.L_gal[~all.all_data.z_assigned_flag], simple_4.all_data.L_gal[~simple_4.all_data.z_assigned_flag]])


## Galaxy Neighborhood Examiner

In [ ]:
#pp.add_halo_columns(mxxl_simple_4)

In [ ]:
data = mxxl_simple_4.all_data

In [ ]:
lost_galaxies = data.loc[data['z_assigned_flag'] == 1]
#lost_galaxies_2 = nn.all_data.loc[nn.all_data['fiber_assigned_0'] == 0]
obs_galaxies = data.loc[data['z_assigned_flag'] == 0]
#obs_galaxies_2 = nn.all_data.loc[nn.all_data['fiber_assigned_0'] == 1]
print("Lost galaxies: ", len(lost_galaxies), "Observed Galaxies: ", len(obs_galaxies))

In [ ]:
PLOTS_TO_MAKE = 10
GALAXY_POOL = lost_galaxies

#START_INDEX = 777
#for i in range(START_INDEX, START_INDEX + PLOTS_TO_MAKE):
#    index = lost_galaxies.index[i]
#    examine_around(index)
print("Number of galaxies to choose from: ", len(GALAXY_POOL))
indexes = np.random.randint(0, len(GALAXY_POOL)-1, size=PLOTS_TO_MAKE)
for i in indexes:
    target = GALAXY_POOL.iloc[i]
    pp.examine_around(target, data)

### Idea: analyze entire neighborhood and look for groups of similar z galaxies, choose a z from the biggest group

In [ ]:
def neighbor_exam(target):
    nearby_angle = coord.Angle('5m')
    z_eff = target.z
    target_dist_true = z_to_ldist(target.z_obs)

    ra_max = (coord.Angle(target.RA*u.degree) + nearby_angle).value
    ra_min = (coord.Angle(target.RA*u.degree) - nearby_angle).value
    dec_max = (coord.Angle(target.Dec*u.degree) + nearby_angle).value
    dec_min = (coord.Angle(target.Dec*u.degree) - nearby_angle).value

    nearby = obs_galaxies.query('RA < @ra_max and RA > @ra_min and Dec < @dec_max and Dec > @dec_min')

    close_neighbors = 0
    if len(nearby) > 0:
        close_neighbors = np.isclose(nearby.ldist_true.to_numpy(), target_dist_true, rtol=0.0, atol=20)

    return (np.sum(close_neighbors), len(nearby), np.sum(close_neighbors)/len(nearby))

MAX = 300
close = np.empty(MAX)
total = np.empty(MAX)
frac = np.empty(MAX)
for i in range(0,MAX):
    target = lost_galaxies.iloc[i]
    close[i], total[i], frac[i] = neighbor_exam(target)

In [ ]:
max_finished = 81408
finished_close = close[0:max_finished]
finished_total = total[0:max_finished]
with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_close.npy', 'wb') as f:
    np.save(f, finished_close)
with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_total.npy', 'wb') as f:
    np.save(f, finished_total)

In [ ]:
with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_close.npy', 'rb') as f:
    close = np.load(f)

with open(OUTPUT_FOLDER + 'mxxl_lostgal_neighborhood_total.npy', 'rb') as f:
    total = np.load(f)

frac = close / total

In [ ]:
bins = np.linspace(0,30,31)
trash = plt.hist(close, bins=bins)
plt.title("Lost Galaxies Neighbors at ~Correct z")
plt.xlabel("Count of Similar z Neighbors")
plt.ylabel("Count of Lost Galaxies")
print("Hopeless Fraction: ", np.sum(close==0) / len(close))
print("Essentially Hopeless Fraction: ", (np.sum(close==0) + np.sum(close==1)) / len(close))

In [ ]:
viable = close > 1
frac[viable]
trash=plt.hist(frac[viable], bins=30)
plt.title("Viable Lost Galaxies: Correct z Neighbor Fraction")
plt.xlabel("Fraction with Similar z")
plt.ylabel("Count of Viable Lost Galaxies")

# Tests

In [ ]:
blue_color = GLOBAL_RED_COLOR_CUT - 0.1
red_color = GLOBAL_RED_COLOR_CUT + 0.1

blue_dn = -1
red_dn = 3

results = is_quiescent_BGS_smart(np.array([7,8,9]), np.array([red_dn, np.nan, blue_dn]), np.array([blue_color, blue_color, red_color]))
assert results[0] == True
assert results[1] == False
assert results[2] == False

In [ ]:
# How many halos were assigned below a certain cutoff?
df = simple4_BGS.all_data
M_HALL_CUT = 10**11
small_halo_df = df[df.M_halo < M_HALL_CUT]

print(len(small_halo_df), len(df))

plt.hist(small_halo_df.z, bins=100)
